In [1]:
import os
import math
import pandas as pd
import numpy as np
import urllib
import matplotlib.pyplot as plt
%matplotlib inline

import psycopg2

import datetime

import hydrofunctions as hf

import fbprophet 

# For inputting City information to get distance to sensors.
from  geopy.geocoders import Nominatim

# Date time conversion registration.
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()


In [8]:
## Reading in site data from 'Gage_sites.txt'
file = 'Gage_sites.txt'
sites = pd.read_csv(file, sep = '\t', header = 39, skiprows = [40], dtype = {"site_no" : "str"})

site_no = list(sites["site_no"])
site_loc = sites.filter(['site_no', 'station_nm', 'dec_lat_va', 'dec_long_va'])
site_loc.head()

,site_no,station_nm,dec_lat_va,dec_long_va
0,06614800,"MICHIGAN RIVER NEAR CAMERON PASS, CO",40.496094,-105.865012
1,06620000,"NORTH PLATTE RIVER NEAR NORTHGATE, CO",40.936639,-106.339194
2,06696980,"TARRYALL CREEK AT UPPER STATION NEAR COMO, CO",39.339433,-105.911681
3,06700000,"SOUTH PLATTE RIVER ABOVE CHEESMAN LAKE, CO.",39.162769,-105.310273
4,06701620,"TROUT CREEK BLW FERN CR NR WESTCREEK, CO.",39.167491,-105.122211


In [7]:
site_no.index('394220106431500') ## Has water temp data. Need to extract...

333

In [11]:
start = '2000-01-01'
end = str(datetime.datetime.today().strftime('%Y-%m-%d')) #Gets today's date.

for site in site_no[332:335] :
    herring = hf.NWIS(site, 'dv', start, end)
    herring.get_data()
    my_dict = herring.json()
    df = hf.extract_nwis_df(my_dict)
    df.rename(index=str, columns = {"USGS:"+site+":00060:00003" : "y", 
                                    "USGS:"+site+":00060:00003_qualifiers" : "flags"}, 
             inplace = True)
    df['ds'] = df.index[:]
    print(df.tail())


                        y flags                   ds
datetime                                            
2020-01-22 00:00:00  10.3     P  2020-01-22 00:00:00
2020-01-23 00:00:00  10.2     P  2020-01-23 00:00:00
2020-01-24 00:00:00  10.4     P  2020-01-24 00:00:00
2020-01-25 00:00:00  10.7     P  2020-01-25 00:00:00
2020-01-26 00:00:00  10.9     P  2020-01-26 00:00:00
                      y  flags                   ds
datetime                                           
2020-01-22 00:00:00 NaN  P Ice  2020-01-22 00:00:00
2020-01-23 00:00:00 NaN  P Ice  2020-01-23 00:00:00
2020-01-24 00:00:00 NaN  P Ice  2020-01-24 00:00:00
2020-01-25 00:00:00 NaN  P Ice  2020-01-25 00:00:00
2020-01-26 00:00:00 NaN  P Ice  2020-01-26 00:00:00
                         y flags                   ds
datetime                                             
2019-10-27 00:00:00   4.33     P  2019-10-27 00:00:00
2019-10-28 00:00:00  12.50     P  2019-10-28 00:00:00
2019-10-29 00:00:00  11.50     P  2019-10-29 00:0

In [14]:
df.iloc[0:3]

,y,flags,ds
datetime,,,
2012-03-01 00:00:00,5.45,A,2012-03-01 00:00:00
2012-03-02 00:00:00,7.81,A,2012-03-02 00:00:00
2012-03-03 00:00:00,6.88,A,2012-03-03 00:00:00


In [16]:
df.plot()
plt.xlabel('Date')
plt.yscale('log')
plt.ylabel('Discharge (cfs)')
plt.title('Daily Mean Discharge for Roaring Fork River')
plt.savefig('RRFork1.png')
plt.close()

In [8]:
df.rename(index=str, columns = {"USGS:09081000:00060:00003" : "y", 
                                "USGS:09081000:00060:00003_qualifiers" : "flags"}, 
         inplace = True)

df['ds'] = df.index[:]

df.head()

,y,flags,ds
datetime,,,
2000-01-23 00:00:00,252.0,A,2000-01-23 00:00:00
2000-01-24 00:00:00,252.0,A,2000-01-24 00:00:00
2000-01-25 00:00:00,264.0,A,2000-01-25 00:00:00
2000-01-26 00:00:00,271.0,A,2000-01-26 00:00:00
2000-01-27 00:00:00,262.0,A,2000-01-27 00:00:00


In [9]:
df_prophet = fbprophet.Prophet(changepoint_prior_scale=0.05, daily_seasonality=True, interval_width = 0.25)
df_prophet.fit(df)

In [10]:
df_forecast = df_prophet.make_future_dataframe(periods=365 * 1, freq='D')
df_forecast = df_prophet.predict(df_forecast)

In [11]:
df_prophet.plot(df_forecast, xlabel = 'Date', ylabel = 'Discharge (cfs)')
plt.title('Discharge at ROARING FORK RIVER NEAR EMMA, CO')
plt.ylim(55,6000)
plt.yscale('log')
plt.savefig('RRFork1_Pred.png')
plt.close()

In [12]:
df_prophet.plot_components(df_forecast)
plt.savefig('RRFork1_Components.png')
plt.close()

In [15]:
help(herring)

Help on NWIS in module hydrofunctions.station object:

class NWIS(Station)
 |  A class for working with data from the USGS NWIS service.
 |  
 |  description
 |  
 |  Args:
 |      site (str or list of strings):
 |          a valid site is '01585200' or ['01585200', '01646502']. Site
 |          should be None if stateCd or countyCd are not None.
 |  
 |      service (str):
 |          can either be 'iv' or 'dv' for instantaneous or daily data.
 |          'dv'(default): daily values. Mean value for an entire day.
 |          'iv': instantaneous value measured at this time. Also known
 |          as 'Real-time data'. Can be measured as often as every
 |          five minutes by the USGS. 15 minutes is more typical.
 |  
 |      start_date (str):
 |         should take on the form yyyy-mm-dd
 |  
 |      end_date (str):
 |          should take on the form yyyy-mm-dd
 |  
 |      stateCd (str):
 |          a valid two-letter state postal abbreviation. Default is None.
 |  
 |      county